## YouTube Ranking

In [43]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

- Step 1, 2

In [44]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver = webdriver.Chrome('chromedriver')
driver.get(url)
time.sleep(2)

In [45]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
trs = soup.select('tr.aos-init')
len(trs)

100

- Step 3

In [46]:
tr = trs[0]

# 순위
tr.select_one('.rank').get_text().strip()

'1'

In [47]:
# 카테고리
tr.select_one('.category').get_text().strip()[1:-1]

'음악/댄스/가수'

In [48]:
# 채널명
tr.select_one('.subject a').get_text().strip()

'BLACKPINK'

In [49]:
# 구독자, 조회, 비디오 갯수
subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
view = tr.select_one('.view_cnt').get_text().strip()
video = tr.select_one('.video_cnt').get_text().strip()
subscriber, view, video

('7350만', '233억1663만', '394개')

- Step 4

In [50]:
lines = []
for tr in trs:
    rank = tr.select_one('.rank').get_text().strip()
    category = tr.select_one('.category').get_text().strip()[1:-1]
    subject = tr.select_one('.subject a').get_text().strip()
    subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
    view = tr.select_one('.view_cnt').get_text().strip()
    video = tr.select_one('.video_cnt').get_text().strip()
    lines.append([rank, category, subject, subscriber, view, video])
df = pd.DataFrame(lines, columns = ['순위','카테고리','채널','구독자수','조회수','비디오수'])
df.head()


,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,7350만,233억1663만,394개
1,2,음악/댄스/가수,BANGTANTV,6550만,157억9463만,"1,686개"
2,3,음악/댄스/가수,HYBE LABELS,6500만,221억9069만,818개
3,4,음악/댄스/가수,SMTOWN,3030만,243억1077만,"3,895개"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


In [51]:
# 숫자로 바꿔주는 함수
def convert_unit(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return f'{int(s):,d}'

In [52]:
convert_unit('233억1663만')

'23,316,630,000'

In [53]:
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df.head()

,순위,카테고리,채널,구독자수,조회수,비디오수
0,1,음악/댄스/가수,BLACKPINK,"73,500,000","23,316,630,000",394
1,2,음악/댄스/가수,BANGTANTV,"65,500,000","15,794,630,000","1,686"
2,3,음악/댄스/가수,HYBE LABELS,"65,000,000","22,190,690,000",818
3,4,음악/댄스/가수,SMTOWN,"30,300,000","24,310,770,000","3,895"
4,5,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],"26,500,000","11,052,880,000",223


- Step 6: 모든 페이지

In [54]:
from tqdm.notebook import tqdm

In [55]:
driver = webdriver.Chrome('chromedriver')

In [56]:
lines = []
for page in tqdm(range(1, 11)):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('tr.aos-init')
    for tr in trs:
        rank = tr.select_one('.rank').get_text().strip()
        category = tr.select_one('.category').get_text().strip()[1:-1]
        subject = tr.select_one('.subject a').get_text().strip()
        subscriber = tr.select_one('.subscriber_cnt').get_text().strip()
        view = tr.select_one('.view_cnt').get_text().strip()
        video = tr.select_one('.video_cnt').get_text().strip()
        lines.append([rank, category, subject, subscriber, view, video])

  0%|          | 0/10 [00:00<?, ?it/s]

In [57]:
df = pd.DataFrame(lines, columns = ['순위','카테고리','채널','구독자수','조회수','비디오수'])
for column in df.columns[3:]:
    df[column] = df[column].apply(convert_unit)
df.tail()

,순위,카테고리,채널,구독자수,조회수,비디오수
995,994,애완/반려동물,김메주와 고양이들,"580,000","209,780,000","1,060"
996,997,미분류,일주어터,"580,000","118,380,000",194
997,1001,미분류,쏘대장,"580,000","131,650,000",352
998,1000,영화/만화/애니,네이버 웹툰,"580,000","432,110,000",833
999,999,음악/댄스/가수,DanalEntertainment,"580,000","316,800,000","10,089"


- Step 7

In [58]:
driver.close()

In [59]:
df.to_csv('YouTube랭킹.csv', index=False)